# NLP project: Extracting skills from job descriptions.
# Синтаксический анализ с deeppavlov_syntax_parser + Извлечение навыков через линейные комбинации

In [29]:
import pandas as pd
import numpy as np
from collections import defaultdict
from collections import Counter

import spacy
import itertools
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from wiki_ru_wordnet import WikiWordnet
wwn = WikiWordnet() 

from transformers import AutoModel, AutoTokenizer
from deeppavlov import build_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

import matplotlib as plt

import torch
import wn

In [27]:
wwn = WikiWordnet()

In [55]:
# # тест вызова лемма
# synonyms = set()

# for synset in wwn.get_synsets("управление"):
#     for word_obj in synset.get_words():
#         synonyms.add(word_obj.lemma())  

# print("Синонимы:", list(synonyms))

Синонимы: ['управление', 'руководство', 'администрирование']


In [93]:
class EnhancedSyntaxAnalyzer:
    def __init__(self):
        PROFESSIONAL_POS_TAG = 'PROSKILL'  # тег для профессиональных терминов (и составных, и не составных)
        self.mlm_model = AutoModel.from_pretrained('./mlm_results/final_model')
        self.mlm_tokenizer = AutoTokenizer.from_pretrained('./mlm_results/final_model')
        self.syntax_parser = build_model("syntax_ru_syntagrus_bert", download=True)       
        self.base_skills, self.compound_skills = self._load_skills()   
        self.spacy_nlp = spacy.load("ru_core_news_sm")
        self.syntax_cache = defaultdict(list)
        self.morph = MorphAnalyzer()
        self.stop_words = set(stopwords.words("russian")) # Скачиваем русские стоп-слова (для очистки текста)

    def _load_skills(self):
        try:
            df = pd.read_excel('skills_1706.xlsx')
            base_skills = set(df.query('compound_skill != 1')['skills'])
            compound_skills = set(df.query('compound_skill == 1')['skills'])
            return base_skills, compound_skills
        except Exception as e:
            print(f"Ошибка при чтении файла компаунд-навыков: {e}")
            return set()
    
    def get_token_embeddings(self, text):
        try:
        # Токенизация с получением offset_mapping
            inputs = self.mlm_tokenizer(
                text,
                return_tensors="pt",
                return_offsets_mapping=True,
                add_special_tokens=False,
                truncation=True
            )
        
        # Отделяем offset_mapping, который не нужен для модели
            offset_mapping = inputs.pop('offset_mapping')[0].numpy()
        
        # Получаем эмбеддинги от модели
            with torch.no_grad():
                outputs = self.mlm_model(**inputs)       
            token_embeddings = outputs.last_hidden_state[0]
        
        # Группируем токены в слова
            word_embeddings = []
            current_word = []
        
            for idx, (start, end) in enumerate(offset_mapping):
                if start == end:  # Специальные токены
                    continue
                
            # Получаем текст токена для проверки пунктуации
                token_text = self.mlm_tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][idx].item())
            
            # Обработка пунктуации (добавлено в этом месте)
                if token_text in {':', ',', '.', ';'}:
                # Добавляем текущее слово, если оно есть
                    if current_word:
                        avg_emb = np.mean([t['embedding'] for t in current_word], axis=0)
                        word_embeddings.append({
                            'start': current_word[0]['start'],
                            'end': current_word[-1]['end'],
                            'embedding': avg_emb
                        })
                        current_word = []
                
                # Добавляем пунктуацию как отдельный токен
                    word_embeddings.append({
                        'start': start,
                        'end': end,
                        'embedding': token_embeddings[idx].numpy()
                    })
                    continue
                
            # Обработка обычных слов
                if current_word and start > current_word[-1]['end']:
                # Сохраняем собранное слово
                    avg_emb = np.mean([t['embedding'] for t in current_word], axis=0)
                    word_embeddings.append({
                        'start': current_word[0]['start'],
                        'end': current_word[-1]['end'],
                        'embedding': avg_emb
                    })
                    current_word = []
            
                current_word.append({
                    'start': start,
                    'end': end,
                    'embedding': token_embeddings[idx].numpy()
                })
        
        # Добавляем последнее слово
            if current_word:
                avg_emb = np.mean([t['embedding'] for t in current_word], axis=0)
                word_embeddings.append({
                    'start': current_word[0]['start'],
                    'end': current_word[-1]['end'],
                    'embedding': avg_emb
                })
        
            return word_embeddings
        
        except Exception as e:
            print(f"Error in get_token_embeddings: {str(e)}")
            return []
  
    def normalize_term(self, term):
        term = term.lower()
        replacements = {
            'битрикс24': 'bitrix24',
            'в2в': 'b2b',
            'б2б': 'b2b',
            '1с': '1c',
            'в2с': 'b2c',
            'crm': 'crm',
            '1с': '1c',
            'в2f': 'b2f',
            'б2б': 'b2b'
        }
        return replacements.get(term, term) 


    def preprocess_text(self, text):
        # Очистка текста и нормализация    
        text = re.sub(r'(?<!\S)\d+(?:\.\d+)?(?!\S)', '', text)  # Удаление отдельно стоящих цифр (включая числа с точкой)
        text = re.sub(r'([а-яё])([А-ЯЁ])', r'\1 \2', text)  # Разделяем CamelCase
        text = re.sub(r'([а-яёa-zА-ЯЁA-Z])([()])', r'\1 \2', text)  # Отделяем буквы от скобок
        text = re.sub(r'([()])([а-яёa-zА-ЯЁA-Z])', r'\1 \2', text)  # Отделяем скобки от букв  
        text = re.sub(r'<[^>]+>', '', text)  # Удаляем HTML-теги 
        text = re.sub(r'\n|\t|•|\*', ' ', text)  # Удаляем переводы строк, табуляторы и маркеры списка
        text = re.sub('ё', 'е', text)  # Замена буквы ё на е
        text = re.sub(r'([.,;:])\s*-', r'\1', text)  # Убираем тире после знаков препинания
        text = re.sub(r'\s-', '-', text)  # Убираем пробел перед тире
        text = re.sub(r'-\s', '-', text)  # Убираем пробел после тире
        text = re.sub(r'([а-яёa-zА-ЯЁA-Z])(-\s*)([а-яёa-zА-ЯЁA-Z])', r'\1-\3', text)  # Склеиваем слова с дефисом
        text = re.sub(r'[^\w\s.%&,/\-{}$$:;\']+', '', text)  # Оставляем нужные символы
        text = re.sub(r'\s+', ' ', text) #удаление возможных двойных пробелов после удаления цифр
        text = text.lower().strip()

        #Удаление стоп-слов
        words = text.split()
        filtered_words = [w for w in words if w not in self.stop_words]
        text = ' '.join(filtered_words)

        # Замена профессиональных терминов
        for term in sorted(self.base_skills, key=len, reverse=True):
            text = re.sub(rf'\b{re.escape(term)}\b', self.normalize_term(term), text, flags=re.IGNORECASE)

        return text
        
    def merge_compounds(self, parsed_data):
   
        if not parsed_data:
            return []

        i = 0
        n = len(parsed_data)

        while i < n - 1:
            current = parsed_data[i]
            next_item = parsed_data[i+1] if i+1 < n else None

            # Проверяем составные навыки
            for length in range(min(3, n-i), 1, -1):
                phrase = ' '.join([parsed_data[j]['word'].lower() for j in range(i, i+length)])

                # Проверяем, что фраза находится в compound_skills И НИ ОДИН из токенов не является базовым навыком
                if (phrase in self.compound_skills and 
                    not any(parsed_data[j]['word'].lower() in self.base_skills for j in range(i, i+length))):

                    # Объединяем токены, если образуется составной навык
                    start_pos = parsed_data[i]['start']
                    end_pos = parsed_data[i+length-1]['end']

                    parsed_data[i] = {
                        **current,
                        'word': '_'.join([parsed_data[j]['word'] for j in range(i, i+length)]),
                        'embedding': np.mean([parsed_data[j]['embedding'] for j in range(i, i+length)], axis=0),
                        'is_compound': True,
                        'start': start_pos,
                        'end': end_pos,
                        'pos': 'PROSKILL'
                    }

                # Удаляем объединенные токены
                    for _ in range(length-1):
                        del parsed_data[i+1]
                    n -= (length - 1)
                    break

            i += 1

        return parsed_data

    def parse_sentence_dependencies(self, sentence):
        doc = self.spacy_nlp(sentence)
        return [(w.text, w.dep_, w.head.text) for w in doc]
    def analyze_large_contexts(self, sentences):
    # Генерация биграмм (пар соседних предложений)
        bigrams = list(zip(sentences[:-1], sentences[1:]))
    # Генерация триграмм (тройки соседних предложений)
        trigrams = list(zip(sentences[:-2], sentences[1:-1], sentences[2:]))
        return bigrams, trigrams


    def calculate_frequency_features(self, tokens):
        document = ' '.join(tokens)    
        self.vectorizer = CountVectorizer(
            ngram_range=(1, 1),
            token_pattern=r'(?u)\b\w+\b'
        )   
        freq_matrix = self.vectorizer.fit_transform([document])
        return dict(zip(
            self.vectorizer.get_feature_names_out(),
            freq_matrix.toarray()[0]
    )) 
        
    def extract_significant_patterns(self, data):
        significant_patterns = []
        for entry in data:
            sentence = entry["sentence"]
            doc = self.spacy_nlp(sentence)
            for token in doc:
            # Расширяем список зависимостей
                if token.dep_ in {"obj", "obl", "nsubj", "attr", "advcl", "xcomp", "acl", "amod", "compound", "conj",  "pobj"}:                                
                    pattern = token.text
                    significant_patterns.append(pattern)
        return significant_patterns 
        
    def enhanced_parse_with_features(self, text):
        try:
        #Предварительная обработка
            processed_text = self.preprocess_text(text)
        
        #Получение эмбеддингов
            word_embeddings = self.get_token_embeddings(processed_text)
        
        #Анализ через spaCy
            doc = self.spacy_nlp(processed_text)
            spacy_deps = [(token.text, token.dep_, token.head.text) for token in doc]
        
        #Анализ через DeepPavlov
            syntax_result = self.syntax_parser([processed_text])[0]
            lines = [line for line in syntax_result.strip().split('\n') 
                    if line and not line.startswith('#')]
        
        #Сопоставление данных
            parsed_data = []
            text_pos = 0
        
            for line in lines:
                parts = line.split('\t')
                if len(parts) < 8:
                    continue
                
                word = parts[1]
                word_start = processed_text.find(word, text_pos)
                if word_start == -1:
                    continue
                
                word_end = word_start + len(word)
                text_pos = word_end
            
            # Поиск соответствующего эмбеддинга
                embedding = next(
                    (emb['embedding'] for emb in word_embeddings
                    if emb['start'] <= word_start and emb['end'] >= word_end
                ), None)
            
                parsed_data.append({
                    'id': int(parts[0]),
                    'word': word,
                    'lemma': parts[2],
                    'pos': parts[3],
                    'features': parts[5],
                    'head': int(parts[6]),
                    'deprel': parts[7],
                    'embedding': embedding,
                    'is_compound': False,
                    'spacy_dep': next((d for d in spacy_deps if d[0] == word), None),
                    'start': word_start,
                    'end': word_end
                })
        
        # Корректировка пунктуации
            for i, token in enumerate(parsed_data):
                if token['word'] in {':', ',', '.', ';'} and i > 0:
                    token['head'] = parsed_data[i-1]['id']
                    token['deprel'] = 'punct'


            # Заполнение POS-тегов через spaCy
            doc = self.spacy_nlp(processed_text)
            for token in parsed_data:
                # Ищем соответствующий токен в spaCy по позиции start
                spacy_token = next(
                    (t for t in doc if t.idx == token['start']), 
                    None
                )
                if spacy_token:
                    token['pos'] = spacy_token.pos_  # Заполняем только POS-тег

            # Упрощённая лемматизация через единый метод
            for token in parsed_data:
                token['lemma'] = self.get_lemma(token['word'], token.get('pos'))
            
                # Для отладки (можно удалить)
                if token['word'].lower() == 'управление':
                    print(f"Лемма для 'управление': {token['lemma']}")
                
        #Объединение составных терминов
            parsed_data = self.merge_compounds(parsed_data)

            #Разметка профессиональных терминов
            for token in parsed_data:
            # Проверяем как оригинальные термины, так и нормализованные варианты
                normalized_word = self.normalize_term(token['word'].lower())
                if (token['word'].lower() in self.base_skills or 
                    normalized_word in self.base_skills):
                    token['pos'] = 'PROSKILL'
                    
       # Добавление признаков
            for token in parsed_data:
                token['syntax_features'] = {
                    'head_position': token['head'] - token['id'],
                    'is_root': token['head'] == 0,
                    'dependency_depth': self._calculate_depth(parsed_data, token['id'])
                }
            
                if token['embedding'] is not None:
                    norm = np.linalg.norm(token['embedding'])
                    token['normalized_embedding'] = token['embedding'] / norm if norm > 0 else token['embedding']
        
        # Формирование результата
            return {
                'tokens': parsed_data,
                'spacy_doc': doc,
                'embeddings': word_embeddings,
                'dependencies': spacy_deps,
                'freq_features': self.calculate_frequency_features([t.text for t in doc]),
                'bigrams': list(zip(doc[:-1], doc[1:])),
                'trigrams': list(zip(doc[:-2], doc[1:-1], doc[2:])),
                'compound_skills': [t for t in parsed_data if t.get('is_compound', False)]
            }
        
        except Exception as e:
            print(f"Error in enhanced_parse_with_features: {str(e)}")
            return None

    def _calculate_depth(self, parsed_data, token_id):
        depth = 0
        while True:
            token = next((t for t in parsed_data if t['id'] == token_id), None)
            if token is None or token['head'] == 0:
                break
            depth += 1
            token_id = token['head']
        return depth
        
    def get_lemma(self, word: str, pos: str = None) -> str:
        word = str(word).strip()
        if not word:
            return word

        # Пропускаем специальные токены
        if pos in ['PUNCT', 'NUM', 'X']:
            return word.lower()

        # Обработка составных слов
        if '_' in word:
            parts = word.split('_')
            return '_'.join([self.get_lemma(part, pos) for part in parts])

        # Нормализация профессиональных терминов
        normalized = self.normalize_term(word.lower())
        if normalized != word.lower():
            return normalized

        # Для существительных пробуем WordNet
        if pos in [None, 'NOUN', 'PROSKILL']:
            try:
                synsets = self.wiki_wordnet.get_synsets(normalized)
                if synsets:
                    first_lemma = synsets[0].get_words()[0].lemma()
                    if first_lemma:
                        return first_lemma.lower()
            except:
                pass

        # Основная лемматизация через pymorphy2
        try:
            parsed = self.morph.parse(word)
            if not parsed:
                return word.lower()
            
            # Учитываем часть речи при лемматизации
            return parsed[0].normal_form if pos is None else \
                   next((p.normal_form for p in parsed if p.tag.POS == pos), 
                        parsed[0].normal_form)
        except:
            return word.lower()

# Извлечение навыков через линейные комбинации

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
from string import punctuation
from nltk.corpus import wordnet
from itertools import chain
import re
from string import punctuation, digits

In [73]:
class SkillExtractor:
    def __init__(self, analyzer=None, weights=None, enable_synonyms=True):

        self.analyzer = analyzer or EnhancedSyntaxAnalyzer()
        self.enable_synonyms = enable_synonyms  
        if self.enable_synonyms:
            self.wiki_wordnet = WikiWordnet()
            self._synonyms_cache = {}
            
        self.wiki_wordnet = WikiWordnet()
        self._synonyms_cache = {}
        
        # Настройка весов признаков
        # Валидация весов
        if weights:
            if not all(0 <= v <= 1 for v in weights.values()):
                raise ValueError("Веса должны быть в диапазоне [0, 1]")
            if not np.isclose(sum(weights.values()), 1.0):
                raise ValueError("Сумма весов должна быть равна 1")
        self.weights = weights or {'semantic': 0.6, 'syntax': 0.3, 'position': 0.1}
        
        # Кеширование базовых навыков
        self._base_skills = None
      #  self.pca = PCA(n_components=50) пока не использую
        
    @property
    def base_skills(self):
        """Ленивая загрузка base_skills"""
        if self._base_skills is None:
            self._base_skills = self.analyzer.base_skills
        return self._base_skills

    # Метод создания комбинированного представления токена
    def create_combined_representation(self, token, total_tokens):
    # Проверяем наличие normalized_embedding
        if "normalized_embedding" not in token:
            raise ValueError("Токен не содержит normalized_embedding")

    # Получаем синтаксические признаки (корректно из syntax_features)
        syntax_features = [
            float(token["syntax_features"]["is_root"]),
            float(token["syntax_features"]["dependency_depth"]),
            float(token["syntax_features"]["head_position"])
        ]

    # Позиционная характеристика (индекс берем из id токена)
        position_feature = [token["id"] / total_tokens]

    # Комбинируем признаки с весами
        combined_vector = np.concatenate((
            token["normalized_embedding"] * self.weights['semantic'],
            np.array(syntax_features) * self.weights['syntax'],
            np.array(position_feature) * self.weights['position']
        ))

    # Нормализация
        norm = np.linalg.norm(combined_vector)
        return combined_vector / norm if norm > 0 else combined_vector

    # Основной метод извлечения навыков
    def extract_skills(self, text, min_confidence=None):
        if not text or not isinstance(text, str):
            return {'skills': [], 'confidence_scores': []}
    
        try:
            analysis_result = self.analyzer.enhanced_parse_with_features(text)
            if not analysis_result:
                return {'skills': [], 'confidence_scores': []}
        except Exception as e:
            print(f"Ошибка анализа текста: {str(e)}")
            return {'skills': [], 'confidence_scores': []}

        # Обработка токенов
        skills_data = self._process_tokens(analysis_result['tokens'])
    
        if not skills_data['skills']:
            return {'skills': [], 'confidence_scores': []}

        # Нормализация confidence scores [0, 1]
        confidences = skills_data['confidences']
        if np.max(confidences) > 0:
            confidences = confidences / np.max(confidences)
    
        # Автоматический расчет порога
        if min_confidence is None:
            min_confidence = self.calculate_auto_threshold(confidences)
    
        # Фильтрация
        valid_indices = np.where(confidences >= min_confidence)[0]
        filtered_skills = [skills_data['skills'][i] for i in valid_indices]
        filtered_confidences = confidences[valid_indices]
    
        return {
            'skills': self.postprocess_skills({
                'raw_skills': filtered_skills,
                'confidences': filtered_confidences,
                'original_text': text
            }),
            'confidence_scores': filtered_confidences
        }

    def _process_tokens(self, tokens):
        skills = []
        representations = []
        confidences = []
    
        for token in tokens:
            if not self._is_relevant_token(token):
                continue
            
            rep = self.create_combined_representation(token, len(tokens))
            confidence = self._calculate_confidence(rep, token)
        
            skills.append({
                'text': token['word'],
                'is_compound': token.get('is_compound', False),
                'pos': token['pos'],
                'embedding': rep
            })
            representations.append(rep)
            confidences.append(confidence)
    
        return {
            'skills': skills,
            'confidences': np.array(confidences),
            'representations': representations
        }

    def _is_relevant_token(self, token):
       # """Проверка релевантности токена"""
        return token['pos'] in ['PROSKILL', 'NOUN'] and 'normalized_embedding' in token

    def _calculate_confidence(self, representation, token):
      #  """Вычисление уверенности с кешированием"""
        pos_weight = 1.5 if token['pos'] == 'PROSKILL' else 1.0
        return np.linalg.norm(representation) * pos_weight

    def postprocess_skills(self, data):
        if not data['raw_skills']:
            return []

        # Дедупликация
        unique_skills = {}
        for skill, conf in zip(data['raw_skills'], data['confidences']):
            skill_text = skill['text'].lower()
            if skill_text not in unique_skills or conf > unique_skills[skill_text]['confidence']:
                skill['confidence'] = conf
                unique_skills[skill_text] = skill

        # Валидация
        base_skills_set = {s.lower() for s in self.base_skills}
        validated = []
        
        for skill in unique_skills.values():
            if (skill['text'].lower() in base_skills_set or 
                skill.get('is_compound', False)):
                
                validated.append(skill)
                
                # Добавляем синонимы только если флаг включен
                if self.enable_synonyms and skill['text'].lower() in base_skills_set:
                    for synonym in self._get_cached_synonyms(skill['text']):
                        if synonym != skill['text'].lower():
                            new_skill = skill.copy()
                            new_skill['text'] = synonym
                            new_skill['is_synonym'] = True
                            validated.append(new_skill)

        return sorted(validated, key=lambda x: -x['confidence'])   
         
    def calculate_auto_threshold(self, confidences):
    # """Автоматический расчет порога уверенности по методу межквартильного размаха"""
        if not len(confidences):
            return 0.0
    
        confidences = np.array(confidences)
        q1 = np.percentile(confidences, 25)
        q3 = np.percentile(confidences, 75)
        iqr = q3 - q1
        return max(0.1, q1 - 1.5 * iqr)  # Нижняя граница 0.1

    def get_synonyms(self, word: str) -> list:
        normalized = self.normalize_term(word.lower())
        synonyms = set()
    
        try:
            # Способ 1: Через официальное API
            for synset in self.wiki_wordnet.get_synsets(normalized):
                for word_obj in synset.get_words():
                    try:
                        # Правильное получение леммы через вызов метода
                        lemma = word_obj.lemma().lower()
                        synonyms.add(lemma)
                    except AttributeError:
                        # Если метод lemma() не доступен, пытаемся получить данные через _data
                        if hasattr(word_obj, '_data'):
                            lemma = word_obj._data.get('lemma', '').lower()
                            if lemma:
                                synonyms.add(lemma)
        
            # Способ 2: Через прямое обращение к данным synset (дополнительная проверка)
            for synset in self.wiki_wordnet.get_synsets(normalized):
                if hasattr(synset, '_data'):
                    for word_data in synset._data.get('words', []):
                        if isinstance(word_data, dict):
                            lemma = word_data.get('lemma', '').lower()
                            if lemma:
                                synonyms.add(lemma)
    
        except Exception as e:
            print(f"Ошибка при поиске синонимов для '{word}': {str(e)}")
    
        # Всегда включаем нормализованную форму исходного слова
        synonyms.add(normalized)
    
        # Фильтрация и сортировка результатов
        filtered = [s for s in synonyms if s and len(s) > 1]  # Игнорируем пустые и слишком короткие
        return sorted(filtered, key=lambda x: (x != normalized, x))  # Исходное слово первое

    def _expand_compound_synonyms(self, word: str) -> list:
        # """Отдельный метод для обработки составных терминов"""
        if '_' not in word:
            return self.get_synonyms(word)
            
        components = word.split('_')
        synonyms = set()
        for comp in components:
            synonyms.update(self.get_synonyms(comp))
        return sorted(synonyms)

    def expand_with_synonyms(self, skill_data: dict) -> list:
        # Основной метод расширения навыков синонимами.
        expanded = []
        
        # Базовые синонимы
        base_synonyms = self._get_cached_synonyms(skill_data['text'])
        
        for synonym in base_synonyms:
            new_skill = skill_data.copy()
            new_skill['text'] = synonym
            new_skill['is_synonym'] = (synonym != skill_data['text'])
            expanded.append(new_skill)
        
        return expanded

    def _get_cached_synonyms(self, term: str) -> list:
        #Кеширующий метод для получения синонимов
        normalized = self.analyzer.normalize_term(term.lower())
        
        if normalized not in self._synonyms_cache:
            self._synonyms_cache[normalized] = self._fetch_raw_synonyms(normalized)
            
        return self._synonyms_cache[normalized]

    def _fetch_raw_synonyms(self, term: str) -> list:
        #Низкоуровневое получение синонимов из WordNet
        synonyms = set()
        
        try:
            for synset in self.wiki_wordnet.get_synsets(term):
                for word_obj in synset.get_words():
                    lemma = word_obj.lemma().lower() if callable(word_obj.lemma) else str(word_obj.lemma).lower()
                    if lemma:
                        synonyms.add(lemma)
        except Exception as e:
            print(f"Synonym lookup error for {term}: {str(e)}")
        
        synonyms.add(term)  # Всегда включаем исходный термин
        return sorted(s for s in synonyms if s and len(s) > 1)

    def plot_skills_confidence(self, skills_data):
       # """Визуализация уверенности в навыках"""
        import matplotlib.pyplot as plt
    
        skills = [s['text'] for s in skills_data['skills']]
        confidences = skills_data['confidence_scores']
    
        plt.figure(figsize=(10, 6))
        plt.barh(skills, confidences)
        plt.xlabel('Уверенность')
        plt.title('Распределение уверенности в навыках')
        plt.show()

In [192]:
golden_dataset = pd.read_excel('golden_dataset_1706.xlsx')

In [193]:
golden_dataset.shape

(427, 5)

In [148]:
from sklearn.metrics import precision_recall_fscore_support
from collections import defaultdict

In [149]:
golden_dataset.head()

,job_requirements,description_skills,description_bio,num_skills
0,требования: грамотная устная речь ответственно...,грамотная устная речь; ответственность,требования O : O грамотная B-SKILL устная I-SK...,2
1,требования: желательно знание 1с условия: граф...,знание 1с,требования O : O желательно O знание B-SKILL 1...,1
2,требования: образование не ниже среднего. акти...,активный пользователь пк,требования O : O образование O не O ниже O сре...,1
3,"требования: -внимательность, ответственность; ...",внимательность; ответственность; уверенный пол...,"требования O : O - O внимательность B-SKILL , ...",3
4,требования: грамотная речь.,грамотная речь,требования O : O грамотная B-SKILL речь I-SKIL...,1


In [150]:
# 1. Нормализация навыков в description_skills (ground truth)
def normalize_skills(skills_str):
    skills = []
    for skill in skills_str.split(';'):
        skill = skill.strip()
        if not skill:
            continue
        # Заменяем пробелы на _ в многословных навыках
        if len(skill.split()) > 1:
            skill = skill.replace(' ', '_')
        skills.append(skill.lower())  # Приводим к нижнему регистру
    return skills

In [194]:
golden_dataset['true_skills'] = golden_dataset['description_skills'].apply(normalize_skills)

In [195]:
golden_dataset.head(5)

,Unnamed: 0,job_requirements,description_skills,description_bio,num_skills,true_skills
0,2,требования: грамотная устная речь ответственно...,грамотная устная речь; ответственность,требования O : O грамотная B-SKILL устная I-SK...,2,"[грамотная_устная_речь, ответственность]"
1,6,требования опыт работы знание 1с8.2; уверенный...,опыт работы; знание 1с8.2; уверенный пользоват...,требования O опыт B-SKILL работы I-SKILL знани...,6,"[опыт_работы, знание_1с8.2, уверенный_пользова..."
2,12,требования:образование не ниже среднего профес...,управленческий опыт; опыт в ритейле; пользоват...,требования O : O образование O не O ниже O сре...,3,"[управленческий_опыт, опыт_в_ритейле, пользова..."
3,25,"требования:ответственность, дисциплинированнос...",ответственность; дисциплинированность; внимате...,"требования O : O ответственность B-SKILL , O д...",4,"[ответственность, дисциплинированность, внимат..."
4,28,требования: умение вести переговоры нацеленнос...,умение вести переговоры; нацеленность на резул...,требования O : O умение B-SKILL вести I-SKILL ...,3,"[умение_вести_переговоры, нацеленность_на_резу..."


In [225]:
golden_dataset_test = golden_dataset.iloc[:221,:]
golden_dataset_test.head()

,Unnamed: 0,job_requirements,description_skills,description_bio,num_skills,true_skills
0,2,требования: грамотная устная речь ответственно...,грамотная устная речь; ответственность,требования O : O грамотная B-SKILL устная I-SK...,2,"[грамотная_устная_речь, ответственность]"
1,6,требования опыт работы знание 1с8.2; уверенный...,опыт работы; знание 1с8.2; уверенный пользоват...,требования O опыт B-SKILL работы I-SKILL знани...,6,"[опыт_работы, знание_1с8.2, уверенный_пользова..."
2,12,требования:образование не ниже среднего профес...,управленческий опыт; опыт в ритейле; пользоват...,требования O : O образование O не O ниже O сре...,3,"[управленческий_опыт, опыт_в_ритейле, пользова..."
3,25,"требования:ответственность, дисциплинированнос...",ответственность; дисциплинированность; внимате...,"требования O : O ответственность B-SKILL , O д...",4,"[ответственность, дисциплинированность, внимат..."
4,28,требования: умение вести переговоры нацеленнос...,умение вести переговоры; нацеленность на резул...,требования O : O умение B-SKILL вести I-SKILL ...,3,"[умение_вести_переговоры, нацеленность_на_резу..."


In [197]:
golden_dataset_test.shape

(50, 6)

In [249]:
analyzer = EnhancedSyntaxAnalyzer()
extractor = SkillExtractor(analyzer)

Some weights of the model checkpoint at ./mlm_results/final_model were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./mlm_results/final_model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRA

In [36]:
def predict_skills(text):
    try:
        result = extractor.extract_skills(text)
        
        # Нормализуем предсказания так же, как ground truth
        pred_skills = []
        for skill in result['skills']:
            skill_text = skill['text'].lower().strip()
            
            # Пропускаем пустые навыки
            if not skill_text:
                continue
                
            # Заменяем пробелы на _ в многословных навыках
            if len(skill_text.split()) > 1:
                skill_text = skill_text.replace(' ', '_')
            
            # Добавляем дополнительные нормализации (опционально)
            skill_text = skill_text.replace('ё', 'е')  # Приводим ё → е
            pred_skills.append(skill_text)
            
        return pred_skills
    
    except Exception as e:
        print(f"Ошибка при обработке текста: {str(e)}")
        return []

In [37]:
# Расчет метрик
def evaluate(golden_df):
    results = []
    all_true = []
    all_pred = []
    
    for _, row in golden_df.iterrows():
        true_skills = set(row['true_skills'])
        pred_skills = set(predict_skills(row['job_requirements']))
        
        # Считаем TP, FP, FN
        tp = len(true_skills & pred_skills)
        fp = len(pred_skills - true_skills)
        fn = len(true_skills - pred_skills)
        
        # Precision, Recall, F1
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        results.append({
            'text': row['job_requirements'],
            'true_skills': '; '.join(true_skills),
            'pred_skills': '; '.join(pred_skills),
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'tp': tp,
            'fp': fp,
            'fn': fn
        })
        
        all_true.extend(true_skills)
        all_pred.extend(pred_skills)
    
    # Общие метрики
    avg_precision = np.mean([r['precision'] for r in results])
    avg_recall = np.mean([r['recall'] for r in results])
    avg_f1 = np.mean([r['f1'] for r in results])
    
    # Микро-усредненные метрики
    total_tp = sum(r['tp'] for r in results)
    total_fp = sum(r['fp'] for r in results)
    total_fn = sum(r['fn'] for r in results)
    
    micro_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    micro_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    micro_f1 = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0
    
    return {
        'results': pd.DataFrame(results),
        'macro_metrics': {
            'precision': avg_precision,
            'recall': avg_recall,
            'f1': avg_f1
        },
        'micro_metrics': {
            'precision': micro_precision,
            'recall': micro_recall,
            'f1': micro_f1
        }
    }


In [ ]:
metrics = evaluate(golden_dataset_test)

In [253]:
print("=== Macro Metrics ===")
print(f"Precision: {metrics['macro_metrics']['precision']:.3f}")
print(f"Recall: {metrics['macro_metrics']['recall']:.3f}")
print(f"F1: {metrics['macro_metrics']['f1']:.3f}")

print("\n=== Micro Metrics ===")
print(f"Precision: {metrics['micro_metrics']['precision']:.3f}")
print(f"Recall: {metrics['micro_metrics']['recall']:.3f}")
print(f"F1: {metrics['micro_metrics']['f1']:.3f}")

=== Macro Metrics ===
Precision: 0.789
Recall: 0.729
F1: 0.749

=== Micro Metrics ===
Precision: 0.819
Recall: 0.720
F1: 0.766


In [254]:
# Анализ ошибок
error_analysis = defaultdict(int)
for _, row in metrics['results'].iterrows():
    for skill in set(row['pred_skills'].split('; ')) - set(row['true_skills'].split('; ')):
        error_analysis[skill] += 1

print("\nTop 10 False Positives:")
for skill, count in sorted(error_analysis.items(), key=lambda x: -x[1])[:]:
    print(f"{skill}: {count}")


Top 10 False Positives:
ответственность: 50
приемка_товара: 33
1c: 10
: 8
консультирование: 7
позитивный: 4
выкладка_товара: 3
внимательность: 2
переговоры: 1
высшее_образование: 1
уверенный_пользователь_пк: 1
амбициозный: 1
вежливое_обслуживание_покупателей: 1
исполнительность: 1
эмпатия: 1
желание_зарабатывать: 1
выкладка_товаров: 1
обслуживание_покупателей: 1
контроль_сроков_годности: 1
коммуникативные_навыки: 1
пользователь_пк: 1
оптовые_продажи: 1
работоспособность: 1
autocad: 1
консультация_клиентов: 1
преодоление_возражений: 1
оформление_витрин: 1
сопровождение_клиентов: 1
продажа: 1
компьютер: 1
мерчендайзер: 1
